In [1]:
import tensorflow as tf
import pickle
import numpy as np
import os

CIFAR_DIR = './cifar-10-batches-py'
print(os.listdir(CIFAR_DIR))

['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [2]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./tensorboard --logdir=dir

In [3]:
def load_data(filename):
    """read data from data file."""
    with open (filename, "rb") as f:
        data = pickle.load(f,encoding='bytes')
        return data[b'data'], data[b'labels']
    
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0 ## 遍历起始位置
        if self._need_shuffle:
            self._shuffle_data()
    
    def _shuffle_data(self):
        # [0, 1, 2, 3, 4, 5] -> [5, 3, 2, 4, 0 ,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0 
                end_indicator = batch_size
            else:
                raise Exception("have no more example")
        
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]    
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
    
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

batch_data, batch_labels = train_data.next_batch(10)
print(batch_data, batch_labels)

(50000, 3072) (50000,)
(10000, 3072) (10000,)
[[196 200 204 ... 222 223 225]
 [144 138 137 ... 169 160 137]
 [131 203 237 ...  46  48  46]
 ...
 [197 224 224 ...  86 117 127]
 [118 106 128 ... 191 214 216]
 [ 88  83  78 ... 224 222 218]] [5 6 7 3 1 8 2 2 3 2]


In [4]:
batch_size = 20

x = tf.placeholder(tf.float32, [batch_size, 3072])

# [None], eg: [0,6,5,3]
y = tf.placeholder(tf.int64, [batch_size])

is_training = tf.placeholder(tf.bool, [])

x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

x_image_arr = tf.split(x_image, num_or_size_splits=batch_size, axis=0)
result_x_image_arr = []

for x_single_image in x_image_arr:
    # x_single_image: [1, 32, 32, 3] -> [32, 32, 3]
    x_single_image = tf.reshape(x_single_image, [32, 32, 3])
    data_aug_1 = tf.image.random_flip_left_right(x_single_image)
    data_aug_2 = tf.image.random_brightness(data_aug_1, max_delta=65)
    data_aug_3 = tf.image.random_contrast(data_aug_2, lower=0.2, upper=1.8)
    x_single_image = tf.reshape(data_aug_3, [1, 32, 32, 3])
    result_x_image_arr.append(x_single_image)
result_x_images = tf.concat(result_x_image_arr, axis=0)
normal_result_x_images = result_x_images / 127.5 - 1
    
    
def conv_wrapper(inputs, name, is_training, output_channel=32, kernel_size=(3,3), padding='same', activation=tf.nn.relu):
    """wrapper of tf.layers.conv2d"""
    # without bn: conv -> activation
    # with batch normalization: conv -> bn -> activation
    with tf.name_scope(name):
        conv2d = tf.layers.conv2d(inputs,
                               output_channel,
                               kernel_size,
                               padding=padding,
                               activation=None,
                               name=name + 'conv12d')
        bn = tf.layers.batch_normalization(conv2d, training = is_training)
        return activation(bn)
    
def pooling_wrapper(inputs, name):
    """wrapper of tf.layers.conv2d"""
    return tf.layers.max_pooling2d(inputs,
                                  (2,2),
                                  (2,2),
                                  name=name)

conv1_1 = conv_wrapper(normal_result_x_images, 'conv1_1', is_training)
conv1_2 = conv_wrapper(conv1_1, 'conv1_2', is_training)
conv1_3 = conv_wrapper(conv1_2, 'conv1_3', is_training)
pooling1 = pooling_wrapper(conv1_3, 'pool1')

conv2_1 = conv_wrapper(pooling1, 'conv2_1', is_training)
conv2_2 = conv_wrapper(conv2_1, 'conv2_2', is_training)
conv2_3 = conv_wrapper(conv2_2, 'conv2_3', is_training)
pooling2 = pooling_wrapper(conv2_3, 'pool2')

conv3_1 = conv_wrapper(pooling2, 'conv3_1', is_training)
conv3_2 = conv_wrapper(conv3_1, 'conv3_2', is_training)
conv3_3 = conv_wrapper(conv3_2, 'conv3_3', is_training)
pooling3 = pooling_wrapper(conv3_3, 'pool3')

# [None, 4 * 4 * 32]
flatten = tf.layers.flatten(pooling3)
y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y,logits=y_)
# y_ -> softmax
# y -> one_hot
#loss = ylog_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,0]
correct_prediction = tf.equal(predict, y)
accuracy= tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

W0718 17:33:37.579937 16756 deprecation.py:323] From <ipython-input-4-e9fb6d5f95e1>:39: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0718 17:33:37.591918 16756 deprecation.py:506] From D:\anaconda3\envs\deep_learning\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0718 17:33:38.407915 16756 deprecation.py:323] From <ipython-input-4-e9fb6d5f95e1>:40: batch_normalization (from tensorflow.python.layers.normalization) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE

In [5]:
def variable_summary(var, name):
    """Constructs summary fro statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)

with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')
        
# 1. 指定面板图上显示的变量
loss_summary = tf.summary.scalar('loss', loss)
# 'loss': <10, 1.1>, <20, 1.08>
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

inputs_summary = tf.summary.image('inputs_image', result_x_images)

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

# 2. 训练过程中将这些变量计算出来，输出到文件中
LOG_DIR = '.'
run_label = 'run_vgg_tensorboard'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')

if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [ ]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps =100

output_summary_every_steps = 100

with tf.Session() as sess:
    sess.run(init)
    # 2. 训练过程中将这些变量计算出来，输出到文件中
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data, fixed_test_batch_labels = train_data.next_batch(batch_size)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i+1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)
        
        eval_ops_results = sess.run(
            eval_ops, 
            feed_dict={
                x: batch_data,
                y: batch_labels,
                is_training: True})
        loss_val, accu_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            test_summary_str = sess.run([merged_summary_test],
                                           feed_dict={
                                               x: fixed_test_batch_data,
                                               y: fixed_test_batch_labels,
                                               is_training: False
                                           })[0]
            test_writer.add_summary(test_summary_str, i+1)
        if i % 100 == 0:
            print('[Train] step: %d, loss: %4.5f, acc:%4.5f'% (i, loss_val, accu_val)) 
        if i % 500 == 0:
            test_data = CifarData(test_filenames, False)
            
            all_test_acc_val = []
            
            for j in range(test_steps):
                test_batch_data, test_batch_labels = test_data.next_batch(batch_size)
                test_acc_val = sess.run([accuracy],
                                       feed_dict={
                                           x: test_batch_data,
                                           y: test_batch_labels,
                                           is_training: False
                                       })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

[Train] step: 0, loss: 2.79400, acc:0.20000
(10000, 3072) (10000,)
[Test] Step: 1, acc: 0.08950
[Train] step: 100, loss: 1.91471, acc:0.30000
[Train] step: 200, loss: 2.34174, acc:0.30000
[Train] step: 300, loss: 1.65327, acc:0.35000
[Train] step: 400, loss: 2.00087, acc:0.25000
[Train] step: 500, loss: 1.85610, acc:0.30000
(10000, 3072) (10000,)
[Test] Step: 501, acc: 0.14400
[Train] step: 600, loss: 1.63890, acc:0.35000
[Train] step: 700, loss: 1.58881, acc:0.25000
[Train] step: 800, loss: 1.89728, acc:0.30000
[Train] step: 900, loss: 1.76466, acc:0.40000
[Train] step: 1000, loss: 1.17290, acc:0.60000
(10000, 3072) (10000,)
[Test] Step: 1001, acc: 0.15250
[Train] step: 1100, loss: 0.98209, acc:0.70000
[Train] step: 1200, loss: 1.94285, acc:0.25000
[Train] step: 1300, loss: 1.17774, acc:0.60000
[Train] step: 1400, loss: 1.09331, acc:0.75000
[Train] step: 1500, loss: 1.48338, acc:0.30000
(10000, 3072) (10000,)
[Test] Step: 1501, acc: 0.26100
[Train] step: 1600, loss: 0.92773, acc:0.600

[Test] Step: 14001, acc: 0.18900
[Train] step: 14100, loss: 1.20340, acc:0.60000
[Train] step: 14200, loss: 0.42205, acc:0.90000
[Train] step: 14300, loss: 0.58363, acc:0.80000
[Train] step: 14400, loss: 0.37151, acc:0.80000
[Train] step: 14500, loss: 0.30645, acc:0.90000
(10000, 3072) (10000,)
[Test] Step: 14501, acc: 0.21250
[Train] step: 14600, loss: 1.14112, acc:0.65000
[Train] step: 14700, loss: 0.76709, acc:0.85000
[Train] step: 14800, loss: 0.91343, acc:0.75000
[Train] step: 14900, loss: 0.91947, acc:0.65000
[Train] step: 15000, loss: 0.41005, acc:0.90000
(10000, 3072) (10000,)
[Test] Step: 15001, acc: 0.18750
[Train] step: 15100, loss: 0.73980, acc:0.75000
[Train] step: 15200, loss: 0.40375, acc:0.90000
[Train] step: 15300, loss: 1.05833, acc:0.70000
[Train] step: 15400, loss: 0.45691, acc:0.85000
[Train] step: 15500, loss: 0.27445, acc:0.85000
(10000, 3072) (10000,)
[Test] Step: 15501, acc: 0.17200
[Train] step: 15600, loss: 0.48053, acc:0.75000
[Train] step: 15700, loss: 0.91

[Train] step: 28000, loss: 0.36336, acc:0.90000
(10000, 3072) (10000,)
[Test] Step: 28001, acc: 0.19600
[Train] step: 28100, loss: 0.49907, acc:0.90000
[Train] step: 28200, loss: 0.31167, acc:0.95000
[Train] step: 28300, loss: 0.77479, acc:0.75000
[Train] step: 28400, loss: 0.68345, acc:0.75000
[Train] step: 28500, loss: 0.64338, acc:0.70000
(10000, 3072) (10000,)
[Test] Step: 28501, acc: 0.22700
[Train] step: 28600, loss: 0.52438, acc:0.90000
[Train] step: 28700, loss: 0.30442, acc:0.80000
[Train] step: 28800, loss: 0.43614, acc:0.80000
[Train] step: 28900, loss: 0.44348, acc:0.95000
[Train] step: 29000, loss: 0.40654, acc:0.80000
(10000, 3072) (10000,)
[Test] Step: 29001, acc: 0.20400
[Train] step: 29100, loss: 0.53740, acc:0.75000
[Train] step: 29200, loss: 0.21555, acc:0.95000
[Train] step: 29300, loss: 0.66803, acc:0.80000


In [ ]:
print('[Test] Step: %d, acc: %4.5f'%(i+1, test_acc)) 

In [ ]:
# 命令
# tensorboard --logdir=train:'train',test:'test'